In [1]:
import numpy as np
import os
import cv2 as cv
from keras.models import *
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D
from keras.layers import merge 
from keras.layers import concatenate
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History
from keras import backend as keras

Using TensorFlow backend.


In [2]:
class myUnet:
    def __init__(self, img_rows = 200, img_cols = 200):
        
        self.img_rows = img_rows
        self.img_cols = img_cols
   
    def load_data(self):
        
        #image_dir = 'real_images_ER'
        image_dir = 'images_ER'
        mask_dir = 'masks_ER'
        list_all_img_dir = os.listdir(image_dir)
        img_list = [i for i in list_all_img_dir if i.endswith('.tif')]
        
        ###
        img_numb = len(img_list)
        train_numb = 240
        ###
        
        
        imgs_all = np.empty([img_numb,self.img_rows, self.img_cols,1])
        imgs_mask_all = np.empty([img_numb,self.img_rows, self.img_cols,1])
        
        
        for idx in range(img_numb):
            filename = img_list[idx]
            #print(idx)
            img_path = image_dir+'/'+filename
            img = cv.imread(img_path,0)
            img = np.expand_dims(img,axis=2)
            img = np.expand_dims(img,axis=0)
            imgs_all[idx,:,:,:] = img
            
            mask_path = mask_dir+'/'+filename
            img_mask = cv.imread(mask_path,0)
            img_mask = np.expand_dims(img_mask,axis=2)
            img_mask = np.expand_dims(img_mask,axis=0)
            imgs_mask_all[idx,:,:,:] = img_mask
        print('data dimension:', imgs_all.shape)
        
        img_max_thr = imgs_all.max()
        imgs_all = imgs_all/img_max_thr
        
        imgs_mask_all[imgs_mask_all>0]=1
        imgs_mask_all[imgs_mask_all<=0]=0
        
        imgs_train = imgs_all[:train_numb,:,:,:]
        imgs_mask_train = imgs_mask_all[:train_numb,:,:,:]
        imgs_test = imgs_all[train_numb:,:,:,:]
        imgs_mask_test = imgs_mask_all[train_numb:,:,:,:]
             
        return imgs_train, imgs_mask_train, imgs_test, imgs_mask_test 
    
    def get_unet(self):
        inputs = Input((self.img_rows, self.img_cols,1))
        conv1 = Conv2D(64, kernel_size=(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
            
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
            
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
        drop3 = Dropout(0.2)(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
            
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
        drop4 = Dropout(0.2)(conv4)

        up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop4))
        merge7 = concatenate([conv3,up7], axis = 3) 
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

        up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
        merge8 = concatenate([conv2,up8], axis = 3) 
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

        up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8)) 
        merge9 = concatenate([conv1,up9], axis = 3) 
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv9 = Conv2D(2, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
            
        conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9) 
        model = Model(inputs = inputs, outputs = conv10) 
        model.compile(optimizer = SGD(lr = 0.3), loss = 'binary_crossentropy', metrics = ['accuracy'])
        #model.compile(optimizer = Adam(lr=0.0001,beta_1=0.95, beta_2=0.999), loss = 'binary_crossentropy', metrics = ['accuracy'])    
        return model      
        
    def run_unet(self):
        
        print('Loading data')
        imgs_train, imgs_mask_train, imgs_test, imgs_mask_test = self.load_data()
        print('Loading data done')
        model = self.get_unet()
        print('Load unet model done')
        
        result_folder = 'result_v9'
        
        # Training
        if not os.path.exists(result_folder):
            os.makedirs(result_folder)
      
        model_checkpoint = ModelCheckpoint('./'+result_folder+'/unet.hdf5',monitor='loss',verbose=1,save_best_only=True)
        print('Fitting model...')
        model.fit(imgs_train, imgs_mask_train,batch_size=1,epochs=15,verbose=1,shuffle=True,validation_split=0.1, callbacks=[model_checkpoint])
  
        # Test 
        #print('loading weights')
        #model.load_weights('./trial1_result/unet.hdf5')
        #print('load weights done')
        
        test_evaluation = model.evaluate(imgs_test,imgs_mask_test,batch_size=1,verbose=1)
        print test_evaluation
        test_loss,test_acc = test_evaluation
        print('test_loss:',test_loss)
        print('test_acc:',test_acc)
        
        print('Predict test data')
        test_predict = model.predict(imgs_test,batch_size=1,verbose=1)
        
        np.save('./'+result_folder+'/unet_predict.npy',test_predict)
        np.save('./'+result_folder+'/unet_imgs_test.npy',imgs_test)
        np.save('./'+result_folder+'/unet_imgs_mask_test.npy',imgs_mask_test)


    

In [3]:
myunet = myUnet()
myunet.run_unet()

Loading data
('data dimension:', (258, 200, 200, 1))
Loading data done
Load unet model done
Fitting model...
Train on 216 samples, validate on 24 samples
Epoch 1/15
216/216 [==============================] - 12s 54ms/step - loss: 0.3814 - acc: 0.8449 - val_loss: 0.2788 - val_acc: 0.8990

Epoch 00001: loss improved from inf to 0.38144, saving model to ./result_v9/unet.hdf5
Epoch 2/15
216/216 [==============================] - 9s 42ms/step - loss: 0.2355 - acc: 0.9105 - val_loss: 0.2403 - val_acc: 0.9071

Epoch 00002: loss improved from 0.38144 to 0.23550, saving model to ./result_v9/unet.hdf5
Epoch 3/15
216/216 [==============================] - 9s 42ms/step - loss: 0.2140 - acc: 0.9167 - val_loss: 0.2234 - val_acc: 0.9115

Epoch 00003: loss improved from 0.23550 to 0.21395, saving model to ./result_v9/unet.hdf5
Epoch 4/15
216/216 [==============================] - 9s 42ms/step - loss: 0.2042 - acc: 0.9193 - val_loss: 0.2192 - val_acc: 0.9126

Epoch 00004: loss improved from 0.21395 to 

In [ ]:
imgs_predict = np.load('./'+result_folder+'/unet_predict.npy')
imgs_test = np.load('./'+result_folder+'/unet_imgs_test.npy')
imgs_mask_test = np.load('./'+result_folder+'/unet_imgs_mask_test.npy')

In [5]:
import scipy.io as sio
sio.savemat('./'+result_folder+'/unet_predict.mat', {'img_predict':imgs_predict})
sio.savemat('./'+result_folder+'/unet_imgs_test.mat', {'img_test':imgs_test})
sio.savemat('./'+result_folder+'/unet_imgs_mask_test.mat', {'img_test':imgs_mask_test})